In [1]:
import cudf as pd
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
adv_data = pd.read_csv('data/adv_data.csv')
event_data = pd.read_csv('data/event_data.csv')
users_data = pd.read_csv('data/users_data.csv')

In [3]:
users_data.drop('delete_dtm', inplace=True, axis=1)

In [4]:
event_phones = event_data['user_phone'].to_arrow()

In [5]:
np_test = np.zeros_like(event_phones)

In [6]:
for i, val in enumerate(event_phones):
    np_test[i] = str(val).strip('+')

In [7]:
event_data['user_phone_edited'] = np_test

In [8]:
event_data.drop('user_phone', axis=1, inplace=True)

In [9]:
user_phones = users_data['user_phone'].to_arrow()

In [10]:
np_test = np.zeros_like(user_phones)

In [11]:
for i, val in enumerate(user_phones):
    np_test[i] = str(val).strip('+')

In [12]:
users_data['user_phone_edited'] = np_test

In [13]:
users_data.drop('user_phone', axis=1, inplace=True)

In [14]:
adv_data_plus_user_data = adv_data.merge(users_data, how='inner', on='user_id')

In [15]:
del adv_data, users_data

In [16]:
all_data = adv_data_plus_user_data.merge(event_data, how='inner', on='user_phone_edited')

In [17]:
del event_data

In [18]:
all_data.drop(['id', 'user_id'], inplace=True, axis=1)

In [59]:
chunk_size = 300_000
start_no = 0
end_no = chunk_size
chunk_amount = int(len(all_data) / chunk_size) + 1

In [61]:
for i in range(chunk_amount):
    all_data.iloc[start_no:end_no].to_csv(f'all_data/all_data_{i}.csv')
    start_no += chunk_size
    end_no += chunk_size